In [1]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import dataset
from dataset import CircularLearningRate

from tensorflow import config
assert config.list_physical_devices('GPU')

In [71]:
BATCH_SIZE = 32
MAXLEN = 64

LETTERS_SIZE = len(dataset.letters_table)
NIQQUD_SIZE = len(dataset.niqqud_table)
DAGESH_SIZE = len(dataset.dagesh_table)
SIN_SIZE = len(dataset.sin_table)
KINDS_SIZE = len(dataset.KINDS)

def build_model(EMBED_DIM=90, UNITS=180):

    layer = input_text = keras.Input(batch_shape=(None, MAXLEN), batch_size=BATCH_SIZE)
    
    layer = layers.Embedding(LETTERS_SIZE, EMBED_DIM, input_length=MAXLEN, mask_zero=True)(layer)
    layer = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True), merge_mode='sum')(layer)
    layer = layers.add([layer, 
            layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True), merge_mode='sum')(layer)])
    
    # kinds_layer = layers.LSTM(32)(layer)
    
    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
        # layers.Softmax(name='K')(layers.Dense(KINDS_SIZE)(kinds_layer))
    ]
    model = keras.Model(inputs=[input_text], outputs=outputs)

    jsmodel = model  #  keras.Model(inputs=[input_text], outputs=outputs[:-1])
    # keras.utils.plot_model(model, to_file='model.png')
    return model, jsmodel

model, jsmodel = build_model()

model.summary()
model.save_weights('./checkpoints/uninit')

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 64, 90)       3960        input_8[0][0]                    
__________________________________________________________________________________________________
bidirectional_14 (Bidirectional (None, 64, 180)      390240      embedding_7[0][0]                
__________________________________________________________________________________________________
bidirectional_15 (Bidirectional (None, 64, 180)      519840      bidirectional_14[0][0]           
___________________________________________________________________________________________

In [74]:
def fit(train_validation, scheduler, verbose=1):
    train, valid = train_validation
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    callbacks = []
    if isinstance(scheduler, CircularLearningRate):
        scheduler.set_dataset(train, BATCH_SIZE)
    if scheduler:
        callbacks.append(scheduler)
        
    x  = train.normalized
    vx = valid.normalized
    y  = {'N': train.niqqud, 'D': train.dagesh, 'S': train.sin, 'C': train.normalized } # , 'K': train.kind }
    vy = {'N': valid.niqqud, 'D': valid.dagesh, 'S': valid.sin, 'C': valid.normalized } # , 'K': valid.kind }
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=1, verbose=verbose, callbacks=callbacks)


In [4]:
def load_data(source, maxlen=MAXLEN, validation=0.1):
    filenames = [os.path.join('texts', f) for f in source]
    train, valid = dataset.load_data(filenames, validation, maxlen=maxlen)
    return train, valid

In [9]:
data_rabanit = load_data(['rabanit'])

In [10]:
data_pre_modern = load_data(['pre_modern'])

In [5]:
data_modern = load_data(validation=0.2, source=['modern'])

In [76]:
model.load_weights('./checkpoints/uninit')
history = fit(data_rabanit, scheduler=CircularLearningRate(20e-4, 50e-4, 5e-4))
model.save_weights('./checkpoints/rabanit')

Train on 73800 samples, validate on 8201 samples
73800/73800 [==============================] - 92s 1ms/sample - loss: 0.3587 - N_loss: 0.2799 - D_loss: 0.0675 - S_loss: 0.0113 - N_accuracy: 0.9028 - D_accuracy: 0.9744 - S_accuracy: 0.9971 - val_loss: 0.1335 - val_N_loss: 0.1014 - val_D_loss: 0.0294 - val_S_loss: 0.0028 - val_N_accuracy: 0.9656 - val_D_accuracy: 0.9896 - val_S_accuracy: 0.9994


In [77]:
model.load_weights('./checkpoints/rabanit')
history = fit(data_pre_modern, scheduler=CircularLearningRate(20e-4, 40e-4, 0.1e-4))
model.save_weights('./checkpoints/pre_modern')

Train on 65308 samples, validate on 7257 samples
65308/65308 [==============================] - 85s 1ms/sample - loss: 0.2840 - N_loss: 0.2154 - D_loss: 0.0621 - S_loss: 0.0064 - N_accuracy: 0.9264 - D_accuracy: 0.9747 - S_accuracy: 0.9982 - val_loss: 0.2146 - val_N_loss: 0.1592 - val_D_loss: 0.0499 - val_S_loss: 0.0055 - val_N_accuracy: 0.9465 - val_D_accuracy: 0.9799 - val_S_accuracy: 0.9985


In [79]:
model.load_weights('./checkpoints/pre_modern')
history = fit(data_modern, scheduler=CircularLearningRate(6e-3, 6e-3, 0.4e-3))
history = fit(data_modern, scheduler=CircularLearningRate(1e-3, 6e-3, 0.5e-3))
model.save_weights('./checkpoints/modern')

Train on 14143 samples, validate on 3536 samples
14143/14143 [==============================] - 27s 2ms/sample - loss: 0.2001 - N_loss: 0.1466 - D_loss: 0.0504 - S_loss: 0.0031 - N_accuracy: 0.9500 - D_accuracy: 0.9807 - S_accuracy: 0.9992 - val_loss: 0.1462 - val_N_loss: 0.1061 - val_D_loss: 0.0377 - val_S_loss: 0.0025 - val_N_accuracy: 0.9648 - val_D_accuracy: 0.9861 - val_S_accuracy: 0.9994
Train on 14143 samples, validate on 3536 samples
14143/14143 [==============================] - 29s 2ms/sample - loss: 0.1319 - N_loss: 0.0939 - D_loss: 0.0360 - S_loss: 0.0020 - N_accuracy: 0.9680 - D_accuracy: 0.9864 - S_accuracy: 0.9995 - val_loss: 0.1374 - val_N_loss: 0.0995 - val_D_loss: 0.0355 - val_S_loss: 0.0025 - val_N_accuracy: 0.9677 - val_D_accuracy: 0.9872 - val_S_accuracy: 0.9995


In [ ]:
data_all = load_data(validation=0.1, source=['biblical', 'rabanit', 'pre_modern', 'modern'])
history = fit(data_all, scheduler=CircularLearningRate(0.01e-4, 1e-4, 0.01e-4))

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['D', 'N'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [ ]:
jsmodel.summary()

In [80]:
import tensorflowjs as tfjs
jsmodel.load_weights('./checkpoints/modern')
tfjs.converters.save_keras_model(jsmodel, '.')

In [69]:
model.load_weights('./checkpoints/modern')

def print_predictions(data, k):
    s = slice(k*BATCH_SIZE, (k+1)*BATCH_SIZE)
    batch = data.normalized[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin, actual_kind] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2]),  dataset.from_categorical(prediction[3])]
    [expected_niqqud, expected_dagesh, expected_sin, expected_kind] = [data.niqqud[s], data.dagesh[s], data.sin[s], data.kind[s]]
    actual = dataset.merge(batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = dataset.merge(batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a, actual_kind[i])
        print('רצוי: ', e, expected_kind[i])

print_predictions(data_modern[1], 6)

מצוי:  OOOOOO הַבְּרִיטִית. בְּמֶשֶׁךְ הַשָּׁנִים הָאִיּוּם שֶׁל הַתְקָפוֹת סַיְיבֶּר עַל לַוְויָינִים  4
רצוי:  OOOOOO הַבְּרִיטִית. בְּמֶשֶׁךְ הַשָּׁנִים הָאִיּוּם שֶׁל הַתְקָפוֹת סַייְבֶּר עַל לַוְויָינִים  4
מצוי:  וְיֵשׁ אֵיזֶשֶׁהוּ חוֹסֶר בְּמִישֶׁהוּ שֶׁיּוֹבִיל מַאֲבָק גַּם נֶגֶד הָאוֹיְבִים שֶׁבַּחוּץ. אֲנִי  4
רצוי:  וְיֵשׁ אֵיזֶשֶׁהוּ חוֹסֶר בְּמִישֶׁהוּ שֶׁיּוֹבִיל מַאֲבָק גַּם נֶגֶד הָאוֹיְבִים שֶׁבַּחוּץ. אֲנִי  4
מצוי:  בַּפַּעַם הַשִּׁישִׁית בָּרְצִיפוּת בְּמִפְעָל. סָאסִיץ' כָּבְשָׁה שְׁנֵי שְׁעָרִים בְּמַהֲלַךְ שָׁלָב  4
רצוי:  בַּפַּעַם הַשִּׁישִּׁית בִּרְצִיפוּת בַּמִּפְעָל. סָאסִיץ' כָּבְשָׁה שְׁנֵי שְׁעָרִים בְּמַהֲלַךְ שְׁלַב  4
מצוי:  OOOOOOOOOOOO "זֶה לֹא הֶגְיוֹנִי. אֲנִי לֹא אֲמוּרָה לִהְיוֹת כָּאן. אֲנִי צְרִיכָה  4
רצוי:  OOOOOOOOOOOO "זֶה לֹא הֶגְיוֹנִי. אֲנִי לֹא אֲמוּרָה לִהְיוֹת כָּאן. אֲנִי צְרִיכָה  4
מצוי:  קַיִץ" ( OOOOOO OOOOOO) וּלְהַתְחִיל בִּגְלִישָׁה בְּמִדְרוֹן הַמְּשׁוּפָּע. הַפְּעִילוּת  4
רצוי:  קַיִץ" ( OOOOOO OOOOOO) וּלְהַתְחִיל בִּגְלִי

In [ ]:
shutil.rmtree(os.sep.join([tempfile.gettempdir(), '.tensorboard-info']), ignore_errors=True)
shutil.rmtree('logs', ignore_errors=True)
os.makedirs('logs')
# %tensorboard --logdir logs

In [ ]:
print(data_modern[1].text[0])
print(data_modern[1].text[1])